# Poincaré operators

In this tutorial, we show how Poincaré operators can be used to efficiently solve the mixed form of the Hodge Laplace problem. Letting $P \Lambda^k$ denote the Whitney forms of order $k$, this problem is posed as : find $(v, u) \in P \Lambda^{k - 1} \times P \Lambda^k$ such that
$$
\begin{align*}
	(v, v')_\Omega - (u, dv')_\Omega &= \langle g, v' \rangle, &
	\forall v' &\in P \Lambda^{k - 1}, \\
	(dv, u')_\Omega + (du, du')_\Omega &= \langle f, u' \rangle, &
	\forall u' &\in P \Lambda^k.
\end{align*}
$$

It is assumed that the boundary conditions are natural and homogeneous.

For $k = dim(\Omega)$, this corresponds to the mixed formulation of the Poisson equation that is commonly used to model Darcy flow.

In [this paper](https://arxiv.org/abs/2410.08830), we show how to construct subspaces $\bar{P} \Lambda^k \subseteq P \Lambda^k$ such that $P \Lambda^k = \bar{P} \Lambda^k \oplus d \bar{P} \Lambda^{k - 1}$ and $d$ is invertible on $\bar{P} \Lambda^k$. In turn, the Hodge-Laplace problem can be solved in four sequential steps:
$$
\begin{align}
		(d\bar{v}, d\bar{v}')_\Omega
		&= \langle f, d\bar{v}' \rangle, &
		\forall \bar{v}' &\in \bar{P} \Lambda^{k - 1} \\
		(d\bar{w}_v, d\bar{w}')_\Omega
		&= \langle g, d\bar{w}' \rangle - (\bar{v}, d\bar{w}')_\Omega, &
		\forall \bar{w}' &\in \bar{P} \Lambda^{k - 2} \\
		(d\bar{u}, d\bar{u}')_\Omega
		&= \langle f, \bar{u}' \rangle - (d\bar{v}, \bar{u}')_\Omega, &
		\forall \bar{u}' &\in \bar{P} \Lambda^{k} \\
		(d\bar{v}_u, d\bar{v}')_\Omega
		&= (\bar{v} + d\bar{w}_v, \bar{v}')_\Omega
		- (\bar{u}, d\bar{v}')_\Omega
		- \langle g, \bar{v}' \rangle, & \forall \bar{v}' &\in \bar{P} \Lambda^{k - 1}.
\end{align}
$$
We can then recover the full solution by setting $v = \bar{v} + d \bar{w}_v$ and $u = \bar{u} + d \bar{v}_u$. In this notebook, we show that this four-step solution procedure is significantly faster than a direct solve of the original problem. 

Let's start by importing the necessary Python packages.

In [ ]:
import time

import numpy as np
import scipy.sparse as sps

import pygeon as pg
from pygeon.numerics.differentials import exterior_derivative as diff
from pygeon.numerics.innerproducts import mass_matrix

We then grid the unit cube using a mesh size $h = \frac1{10}$, and construct a `Poincare` object. This object will contain the necessary information to set up the four subproblems.

In [ ]:
h = 1 / 10
dim = 3

# Grid generation
mdg = pg.unit_grid(dim, h)
mdg.compute_geometry()
print(mdg)

# Create the Poincare object
poin = pg.Poincare(mdg)

The output should be similar to:
```
Mixed-dimensional grid. 
Maximum dimension present: 3 
Minimum dimension present: 3 
Size of highest dimensional grid: Cells: 4610. Nodes: 1145
In lower dimensions: 
Total number of interfaces: 0
```

For easy access, we generate random right-hand sides and store them in `f_list`. Moreover, the mass matrix for the $k$-forms are stored as `Mass[k]` for $k \in [0, n]$ and we store each differential operator $d: P \Lambda^k \to P \Lambda^{k + 1}$ in `Diff[k]`. 
This allows for rapid switching between the different Hodge-Laplace problems later on.

In [ ]:
np.random.seed(0)
f_list = [None] * (dim + 1)
f_list[0] = np.random.rand(mdg.num_subdomain_peaks())
f_list[dim - 2] = np.random.rand(mdg.num_subdomain_ridges())
f_list[dim - 1] = np.random.rand(mdg.num_subdomain_faces())
f_list[dim] = np.random.rand(mdg.num_subdomain_cells())


# Assemble mass, differential, and stiffness matrices
Mass = [mass_matrix(mdg, dim - k, None) for k in range(dim + 1)]  # (u, u')
Diff = [diff(mdg, dim - k) for k in range(dim)]  # du
MD = [Mass[k + 1] @ Diff[k] for k in range(dim)]  # (du, v')
Stiff = [Diff[k].T @ MD[k] for k in range(dim)]  # (du, du')
Stiff.append(pg.cell_stiff(mdg))  # The stiffness matrix of the n-forms is zero


# Using the mass matrix of the 0-forms, we subtract the mean
# from the 0-form right-hand side so that it becomes admissible.
f_list[0] -= np.sum(Mass[0] @ f_list[0]) / np.sum(Mass[0])

Since we are interested in comparing solution times, we introduce a small solver function that prints the relevant information.

In [ ]:
def timed_solve(A, b):
    """
    Takes a matrix A and a vector b, solves the system,
    and prints the system size and solution time.
    """
    t = time.time()
    sol = sps.linalg.spsolve(A.tocsc(), b)
    print("ndof: {:>7}, Time: {:>7.2f}s".format(len(b), time.time() - t))

    return sol

We are now ready to set up and solve the Hodge Laplace problem for given value of $k$.

In [ ]:
# Specify the order of u
k = 2

assert k >= 1 and k <= dim  # This is only interesting for k \in [1, n]

# Extract the right-hand sides from the randomly generated distributions
f = f_list[k]
g = f_list[k - 1]

# First, we assemble the original saddle-point problem
saddle_point = sps.block_array(
    [
        [Mass[k - 1], -MD[k - 1].T],
        [MD[k - 1], Stiff[k]],
    ],
    format="csc",
)
LS = pg.LinearSystem(saddle_point, np.hstack((Mass[k - 1] @ g, Mass[k] @ f)))

# Solve the full system
print("Full   |", end="")
full_sol = LS.solve(solver=timed_solve)

# Split the solution into v and u
v_full = full_sol[: g.size]
u_full = full_sol[g.size :]

print("----------------------------------")

# Second, we solve the problem in four steps

# Solve for bar{v}
print("Step 1 |", end="")
rhs_1 = MD[k - 1].T @ f
v_bar = poin.solve_subproblem(k - 1, Stiff[k - 1], rhs_1, solver=timed_solve)

# Solve for bar{w}_v and set v = bar{v} + d bar{w}_v
print("Step 2 |", end="")
if k >= 2:
    rhs_2 = MD[k - 2].T @ (g - v_bar)
    w_v_bar = poin.solve_subproblem(k - 2, Stiff[k - 2], rhs_2, solver=timed_solve)
    v = v_bar + Diff[k - 2] @ w_v_bar
else:  # k = 1.
    # There are no (k - 2)-forms in this case,
    # but we do need to subtract the mean from v.
    print("ndof: 0, Time: 0.00s")
    v = v_bar - np.sum(Mass[0] @ v_bar) / np.sum(Mass[0])

# Solve for bar{u}
print("Step 3 |", end="")
rhs_3 = Mass[k] @ f - MD[k - 1] @ v
u_bar = poin.solve_subproblem(k, Stiff[k], rhs_3, solver=timed_solve)

# Solve for bar{v}_u and set u = bar{u} + d bar{v}_u
print("Step 4 |", end="")
rhs_4 = Mass[k - 1] @ (v - g) - MD[k - 1].T @ u_bar
v_u = poin.solve_subproblem(k - 1, Stiff[k - 1], rhs_4, solver=timed_solve)
u = u_bar + Diff[k - 1] @ v_u

The output should look similar to:
```
Full   |ndof:   16442, Time:    2.23s
----------------------------------
Step 1 |ndof:    5344, Time:    0.12s
Step 2 |ndof:    1144, Time:    0.01s
Step 3 |ndof:    4610, Time:    0.00s
Step 4 |ndof:    5344, Time:    0.11s
```

In [ ]:
# Consistency check
assert np.allclose(v_full, v)
assert np.allclose(u_full, u)